## Install afinn and hisia from PyPI

In [1]:
#!python -m pip install --user afinn hisia

In [2]:
import pandas as pd

from afinn import Afinn
from hisia import Hisia

## Hisia and Afinn

In [3]:
Hisia('jeg elsker det')

Sentiment(sentiment=positive, positive_probability=0.845, negative_probability=0.155)

In [4]:
Hisia('jeg elsker det ikke')

Sentiment(sentiment=positive, positive_probability=0.695, negative_probability=0.305)

In [5]:
Hisia('jeg elsker det slet ikke')

Sentiment(sentiment=negative, positive_probability=0.279, negative_probability=0.721)

In [6]:
Hisia('jeg elsker det slet ikke').explain

{'decision': -0.9486986657196259,
 'intercept': 0.9959111721097562,
 'features': {('elsker', 0.6979194308004358),
  ('ikke', -3.425370797679455),
  ('slet', -2.481326445401143),
  ('slet ikke', -1.307716737138593)}}

In [7]:
afinn = Afinn(language='da', emoticons=True)
afinn.score('jeg elsker det')

2.0

In [8]:
afinn.score('jeg elsker det ikke')

2.0

In [9]:
afinn.score('jeg elsker det slet ikke')

2.0

In [10]:
afinn.scores('jeg elsker det slet ikke')

[2]

### Tweet context:
@HenrikOslo tweets 
> "I Norge vises utenlandske filmer og tv-programmer på originalspråket med norsk tekst på skjermen. Det har gjort oss bedre i engelsk, som i sin tur har ført til at Norge har mer handel med utlandet og dermed bedre økonomi. 
@teksterne @sprakradet @Trinesg"

@kirstenmarie replies 
> "Lur mig, om præcis det samme ikke også gælder for Danmark. Med andre ord: Det er min skyld, at I er så gode til engelsk, folkens. #glorie #hurra #undertekster #undertekstning #sprogdk'"

In [11]:
# remove #hurra. Kirsten comment is positive (:) I think)
kirsten = 'Lur mig, om præcis det samme ikke også gælder for Danmark. Med andre ord: Det er min skyld, at I er så gode til engelsk, folkens'
afinn.scores(kirsten)

[2, -2]

In [12]:
afinn.score(kirsten) # præcis=2, skuld=-2 => 0

0.0

In [13]:
Hisia(kirsten)

Sentiment(sentiment=positive, positive_probability=0.796, negative_probability=0.204)

In [14]:
Hisia(kirsten).explain # debatable but this is what the model learnt 

{'decision': 1.3627933222101034,
 'intercept': 0.9959111721097562,
 'features': {('d:', -0.7156050921422374),
  ('danmark', 0.0505246013812673),
  ('engelsk', -0.752514842976842),
  ('god', 8.10696711640702),
  ('gælde', 0.6065832619650474),
  ('ikke', -3.425370797679455),
  ('ikke gælde', -0.9519030904157224),
  ('lure', -0.3810356168616799),
  ('ord', 0.06294446930482087),
  ('præcis', 3.21636462116231),
  ('præcis samme', -1.5145388685004244),
  ('samme', -0.1477936239995621),
  ('samme ikke', -0.1697967332198241),
  ('skylde', 1.7154996587486273)}}

## Comparing Afinn and Hisia

In [15]:
# Note: Afinn returns 0 if there is no word found, or the words cancel each other

# Data Sources
# Manual Tripadvisor dansk
# twitter search lang:da #lort
#                lang:da ":)", ":(",  "ikke dårligt", "ikke godt"
# - * remove the :)|:( and place sentiment 1 for :) and -1 for :( in twitter
#     remove #hurra replace # and add # sentiment 1   

# - ** Dig og mig ved daggry af Sanne Mink Jensen, Glenn Rintved

columns = ['text','sentiment']
data_set = [
    ('Lur mig, om præcis det samme ikke også gælder for Danmark. Med andre ord: Det er min skyld, '
     'at I er så gode til engelsk, folkens. # [...]', 1),
    ('Så er jeg klar til min halvårlige ‘gløgg’, hvor jeg inviterer alle mine fb-venner ud af ekkokammeret '
     'og hjem til mig. Vi plejer at være 100 og jeg elsker hvert et øjeblik', 1),
    ('I går havde bloggen 10 års jubilæum! Tak til alle jer, der kommer med indspark og søde kommentarer: [...] #', 1),
    ('Vedvarende energi billigere end fossile brændsler inden for ti år, spår FN-rapport #', 1),
    ('Hvad i alverden skal det gøre godt for? Pouting face #dumt #dyrt',-1),
    ('Fantastisk flot billede af DK`s suverænt pæneste “benzinstation”. Ærgerligt vi ikke har flere af dem *',1),
    ('Herligt👍🏼 De gode timer, der trods alt er, kommer ikke dårligt tilbage', 1),
    ('Lad dem bare. Det er tragisk at de ikke har noget bedre at tage sig til *', 1),
    ('Nej, nej, nej...brutal politivold er ikke slut i USA. Det bliver ved...og '
     'ved...og ved, så længe ansvarlige politikere både nationalt og lokalt lader den her slags sadisme foregå *', -1),
    ('Det er der når en eller anden hacker ens computer så man ingen kontrol har, så bliver man nødt til at nulstille *', -1),
    ('Så gerne der blev lagt “bannere” hen over de tomme pladser. En form for tifo etc. Så man i tv ikke ser alle '
     'de tomme pladser. Har set det brugt i USA på store stadions og det hjælper på den tomhed man oplever. '
     'Allerhelst tilskuere tilbage men går jo nok noget tid *', -1),
    ('Endnu et vildt lavpunkt fra Trump, som fuldkommen grundløst bliver ved med at påstå, at kritisk tv-vært '
      'og tidligere republikansk kongresmedlem er morder * Prøv et sekund at forestille dig, at Mette F påstod, '
      'at Hans Engell var morder. Det er så langt ude. Suk.', -1),
    ('De fleste jeg kender og møder har ikke tillid til politiet. Og de har det til fælles, at de engang havde tillid til '
     'politiet, men da de skulle bruge politiet blev de svigtet og dermed opstod mistilliden så.', 0),
    ('Jeg nikkede, og han kyssede mig i panden og strøg mig over håret. Det var okay. Det var helt okay, det hele'
    'Det var ham, der ikke havde passet godt nok pår mig, det var slet ikke min skyld. **', -1),
    ('Shit, Torben! Det ser ikke godt ud, det ser slet ikke godt ud! Er det her en løsning?', -1),
    ('Udemærket hotel. Små værelser men det ved man jo.. Damen der var var utrolig sød og hjælpsom.'
     ' Havde ikke morgenmad med,da jeg syntes det er for dyrt når man ikke spiser så meget.'
     ' Kommer gerne igen', 1),
    ('En skøn rundtur i den gamle landsby fra 1800 tallet. Fine gamle huse, og nogle steder med arbejdende frivillige. '
     'Skønne udskårne dyr i træ i naturlig størrelse og mange levende dyr af gamle racer. Bestemt et besøg værd, også '
     'for børnefamilier.', 1),
    ('Vi tog en tur til landsbyen men det var en kedelig besøg der var ingen aktivitet og det er ret kedeligt at se '
     'gamle huse uden at der er historien med anden en en kort tekst', -1),
    ('Have store forventninger til stedet, hvis mad - desværre - ikke levede op til mine forventninger. Det drejede sig '
     'om konsistens og tørhed samt mængden af fyld, hvortil personalet (desværre) opførte sig meget uprofessionelt'
     'grænsende til arrogant.', -1),
    (' Fatter ikke at man kan gå så højt op i dyr mad.', -1),
    ('Befriende bog, der stiller sig kritisk mod alle de gængse selvhjælpsbøger, og tager fat i det det virkelig handler om; '
     'at være fucking ligeglad med alt det der ikke er i overensstemmelse med vores livs værdier.', 0),
    ('Det her er nyt for mig. Det er stærkt bekymrende, at København slår sig op på at være grøn, men smider '
     'kloakvand i Øresund. Det er uansvarligt og ulækkert. Og naturligvis findes der andre løsninger, selvom '
     'forvaltningen siger de ikke findes.', -1),
    ('Nogen der ved hvorfor #netto spammer mig?', -1),
    ('hvordan fanden har de overhoved fået penge for at lave kino.dk ?! det er jo håbløst at finde noget som helst!', -1),
    ('Det er ikke dårligt tænkt! Det lyder ihvertfald meget nemmere', 1),
    ('Et sted på twitter samles voksne mennesker i forsøg på at overgå hinanden i morsomheder på bekostning af en '
     'navngiven, kendt person. Jeg forstår det ikke. Og nej, jeg deler det ikke. Det er ikke godt for nogen og '
     'vidner om en betændt kultur i en vis branche.', -1), 
    ('Jeg afblæser streamen i aften fordi jeg føler mig virkelig dårlig.. er svimmel og har det generelt skidt med'
     'ingen appetit. Det har været et par gode dage med streamen, men må sgu stå over i dag *',-1),
    ('Uha, det lyder skræmmende at skulle undvære de mange daglige fix, men måske skal jeg bare kaste mig ud i det *', 1),
    ('Lidt skuffende men jeg tror bare at i skal spille lidt flere kampe sammen så i får en form for flow. Jeg tror på jer drenge *',1),
    ('Læs dog hvad der står før du bliver fornærmet. Og vis du er bedre end dem. Det er vel holdet du elsker, ikke fansene?', 0),
    
]

df = pd.DataFrame(data_set, columns=columns)
df['sentiment'] = df['sentiment'].map({1:'positive', 0:'neutral', -1:'negative'})

In [16]:
df['afinn_score'] = df['text'].apply(afinn.score).astype(int)
df['hisia_score'] = df['text'].apply(lambda x: Hisia(x).sentiment)
pd.set_option('display.max_colwidth', 140)
df

,text,sentiment,afinn_score,hisia_score
0,"Lur mig, om præcis det samme ikke også gælder for Danmark. Med andre ord: Det er min skyld, at I er så gode til engelsk, folkens. # [...]",positive,0,"(positive, 0.796, 0.204)"
1,"Så er jeg klar til min halvårlige ‘gløgg’, hvor jeg inviterer alle mine fb-venner ud af ekkokammeret og hjem til mig. Vi plejer at være ...",positive,3,"(positive, 0.804, 0.196)"
2,"I går havde bloggen 10 års jubilæum! Tak til alle jer, der kommer med indspark og søde kommentarer: [...] #",positive,4,"(positive, 0.756, 0.244)"
3,"Vedvarende energi billigere end fossile brændsler inden for ti år, spår FN-rapport #",positive,0,"(positive, 0.545, 0.455)"
4,Hvad i alverden skal det gøre godt for? Pouting face #dumt #dyrt,negative,4,"(negative, 0.334, 0.666)"
5,Fantastisk flot billede af DK`s suverænt pæneste “benzinstation”. Ærgerligt vi ikke har flere af dem *,positive,5,"(positive, 0.905, 0.095)"
6,"Herligt👍🏼 De gode timer, der trods alt er, kommer ikke dårligt tilbage",positive,0,"(positive, 0.848, 0.152)"
7,Lad dem bare. Det er tragisk at de ikke har noget bedre at tage sig til *,positive,0,"(positive, 0.99, 0.01)"
8,"Nej, nej, nej...brutal politivold er ikke slut i USA. Det bliver ved...og ved...og ved, så længe ansvarlige politikere både nationalt og...",negative,-3,"(negative, 0.459, 0.541)"
9,"Det er der når en eller anden hacker ens computer så man ingen kontrol har, så bliver man nødt til at nulstille *",negative,1,"(negative, 0.326, 0.674)"
